# Siniestros Viales en la Ciudad Autónoma de Buenos Aires
### (Homicidos en Accidentes de Tráfico)

Con base en los datos alojados en la página de "Buenos Aires Data" se extraen los datasets de "homicidios" en Excel que contienen las víctimas mortales, lugar de los hechos y los actores involucrados.

Extracción, Transformación y Limpiza de los datos:

#### 1. Descarga de librería necesarias:

In [1]:
import pandas as pd                                                           # Para abrir y transformar los archivos
from urllib.request import urlretrieve                                        # Para leer y guardar los archivos originales
import openpyxl                                                               # Para manipular los archivos en excel

#### 2. Descargar archivo de "homicidios" y guardarlo en la carpeta "Datos_Originales":

In [2]:
# URL del archivo Excel
url_homicidios = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/victimas-siniestros-viales/homicidios.xlsx'

# Ruta local donde se guardará el archivo
Ubicación_homicidios = "Datos_Originales/Homicidios_CABA.xlsx"

# Descargar el archivo desde la URL y guardarlo localmente
urlretrieve(url_homicidios, Ubicación_homicidios)

# Crear un objeto ExcelFile para manejar la estructura del archivo Excel
wb_Homicidios = openpyxl.load_workbook(Ubicación_homicidios, data_only=True)

# Obtener la lista de nombres de hojas
sheet_names = wb_Homicidios.sheetnames

# Imprimir los nombres de las hojas disponibles
print("Hojas disponibles:", sheet_names)

Hojas disponibles: ['HECHOS', 'DICCIONARIO_HECHOS', 'VICTIMAS', 'DICCIONARIO_VICTIMAS', 'clas']


A través de openpyxl, se crea un objeto "ExcelFile" del que se pueden obtener los nombres con el método "Sheetnames" y manipular eficientemente el archivo de excel.

#### 3. Crear Datafrome de la hoja de "Hechos" del archivo de excel "Homicidios":

In [3]:
# Crear un objeto ExcelFile para manejar el archivo Excel
xl = pd.ExcelFile(Ubicación_homicidios)

# crear dataframe
df_homicidios = xl.parse('HECHOS',skiprows=0)

# Reiniciar los índices después de la eliminación
df_homicidios.reset_index(drop=True, inplace=True)

#### 3.1. Revisar la calidad de la información por columnas:

In [4]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Aparentemente solo los campos "Cruce" y "Altura" tienen registros nulos, pero se debe revisar más específicamente el contenido de las demás columnas.

A continuación se revisan los datos nulos de las columnas que se consideran relevantes:

In [5]:
print('Victima:',df_homicidios['VICTIMA'].unique())
print('--------------------------------------------')
print('Acusado:',df_homicidios['ACUSADO'].unique())
print('--------------------------------------------')
print('Comuna:',df_homicidios['COMUNA'].unique()) # Observamos comuna con valores de "0" lo cual no es real
print('--------------------------------------------')
print('pos x:',df_homicidios['pos x'].min()," hasta ",df_homicidios['pos x'].max()) # datos con "." que deben ser revisados
print('pos y:',df_homicidios['pos y'].min()," hasta ",df_homicidios['pos y'].max()) # datos con "." que deben ser revisados
print('--------------------------------------------')
print('Participantes:',df_homicidios['PARTICIPANTES'].unique()) #Se tiene un valor errado 'PEATON_MOTO-MOTO'

Victima: ['MOTO' 'AUTO' 'PEATON' 'SD' 'CARGAS' 'BICICLETA' 'PASAJEROS' 'MOVIL'
 'OBJETO FIJO' 'PEATON_MOTO']
--------------------------------------------
Acusado: ['AUTO' 'PASAJEROS' 'SD' 'OBJETO FIJO' 'CARGAS' 'MOTO' 'MULTIPLE' 'OTRO'
 'BICICLETA' 'TREN']
--------------------------------------------
Comuna: [ 8  9  1 11 15  4  7 12  3 13 14 10  6  2  5  0]
--------------------------------------------
pos x: -58.35608181  hasta  .
pos y: -34.53465378  hasta  .
--------------------------------------------
Participantes: ['MOTO-AUTO' 'AUTO-PASAJEROS' 'MOTO-SD' 'MOTO-PASAJEROS'
 'MOTO-OBJETO FIJO' 'MOTO-CARGAS' 'PEATON-AUTO' 'PEATON-MOTO'
 'PEATON-PASAJEROS' 'AUTO-AUTO' 'AUTO-OBJETO FIJO' 'PEATON-CARGAS'
 'MOTO-MOTO' 'MULTIPLE' 'SD-SD' 'AUTO-CARGAS' 'AUTO-SD' 'MOTO-MOVIL'
 'CARGAS-CARGAS' 'PEATON-SD' 'BICICLETA-CARGAS' 'SD-CARGAS'
 'BICICLETA-PASAJEROS' 'PASAJEROS-PASAJEROS' 'MOVIL-PASAJEROS'
 'CARGAS-OBJETO FIJO' 'PEATON-BICICLETA' 'SD-AUTO' 'CARGAS-AUTO'
 'BICICLETA-AUTO' 'MOTO-BICICLET

#### 3.1.1. Revisar columna de "COMUNA" con Valor "0":

In [6]:
comunas_con_valor_cero = df_homicidios[df_homicidios['COMUNA'] == 0]
comunas_con_valor_cero 

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,...,NaN,NaN,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.,SD-SD,SD,SD


Revisar columnas con Autopista 25 de mayo:

In [7]:
# Filtrar las filas donde 'Dirección Normalizada' contiene 'AUTOPISTA 25'
Autopista25 = df_homicidios[df_homicidios['Dirección Normalizada'].str.contains('AUTOPISTA 25', case=False, na=False)]

# Mostrar el DataFrame resultante
Autopista25.tail()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
450,2019-0033,1,2019-04-18,2019,4,18,01:00:00,1,"AUTOPISTA 25 DE MAYO Y HUERGO, ING. AV.",AUTOPISTA,...,NaN,"HUERGO, ING. AV.","AUTOPISTA 25 DE MAYO y HUERGO, ING. AV.",1,Point (108916.48040002 100856.88142642),-58.36607137,-34.62150748,MOTO-SD,MOTO,SD
555,2020-0035,1,2020-08-04,2020,8,4,05:30:00,5,"AU 25 DE MAYO ALTURA PUMACAHUA Y RECUERO, CASI...",AUTOPISTA,...,NaN,NaN,AUTOPISTA 25 DE MAYO y PUMACAHUA,7,Point (101156.55025059 99210.16912341),-58.45068621,-34.63638927,MOTO-PASAJEROS,MOTO,PASAJEROS
601,2021-0003,1,2021-01-10,2021,1,10,07:40:00,7,"AU 25 DE MAYO KM. 3,7",AUTOPISTA,...,NaN,NaN,AUTOPISTA 25 DE MAYO KM. 3.7,7,Point (103005.63484321 99983.28177809),-58.43052224,-34.62941648,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
631,2021-0033,1,2021-04-14,2021,4,14,17:50:00,17,AU 25 DE MAYO ALTURA BAJADA BOEDO AV.,AUTOPISTA,...,NaN,NaN,AUTOPISTA 25 DE MAYO y BOEDO AV.,5,Point (104341.13684161 100202.74307433),-58.41595919,-34.62743346,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO
680,2021-0082,1,2021-11-06,2021,11,6,04:39:00,4,AU 25 DE MAYO KM7 (ALTURA PASCO),AUTOPISTA,...,NaN,NaN,AUTOPISTA 25 DE MAYO y PASCO,3,Point (106179.07517962 100523.29161457),-58.39591856,-34.62453459,MOTO-AUTO,MOTO,AUTO


debido a que 25 de Mayo de relaciona con varias comunas se imputará el "0" como "SD":

In [8]:
# Imputar "SD" en lugar de "0" en la columna 'COMUNA'
df_homicidios.loc[df_homicidios['COMUNA'] == 0, 'COMUNA'] = 'SD'

comunas_con_valor_cero = df_homicidios[df_homicidios['COMUNA'] == 'SD']
comunas_con_valor_cero 

C:\Users\davin\AppData\Local\Temp\ipykernel_20000\1434196177.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'SD' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_homicidios.loc[df_homicidios['COMUNA'] == 0, 'COMUNA'] = 'SD'


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,SD,Point (. .),.,.,PEATON-SD,PEATON,SD
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,...,NaN,NaN,AUTOPISTA 25 DE MAYO,SD,Point (. .),.,.,SD-SD,SD,SD


#### 3.1.2. Revisar los datos de coordenadas:

In [9]:
# Filtrar las filas donde 'Dirección Normalizada' contiene 'AUTOPISTA 25'
Coordenada_X0 = df_homicidios[df_homicidios['pos x']=='.']

# Mostrar el DataFrame resultante
Coordenada_X0.tail()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,...,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,PASAJEROS
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,...,NaN,NaN,NaN,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,...,NaN,NaN,NaN,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,...,NaN,NaN,MURGUIONDO,9,Point (. .),.,.,PEATON-CARGAS,PEATON,CARGAS
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,...,NaN,NaN,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS


Se imputarán las coordenadas basados en la comuna y tipo de calle, a continuación se realiza la imputación para el eje x:

In [10]:
# Filtrar las filas donde 'pos x' es igual a '.'
Coordenada_X0 = df_homicidios[df_homicidios['pos x'] == '.']

# Obtener un DataFrame de referencia con valores de 'pos x' que no son '.'
Referencia_X0 = df_homicidios[df_homicidios['pos x'] != '.']

# Imputar los valores de 'pos x' basándose en la comuna de la Referencia_X0
def imputar_pos_x(row):
    filtro = (Referencia_X0['COMUNA'] == row['COMUNA'])
    if filtro.any():
        return Referencia_X0[filtro]['pos x'].iloc[0]
    else:
        return row['pos x']

Coordenada_X0['pos x'] = Coordenada_X0.apply(imputar_pos_x, axis=1)

# Actualizar los valores en el DataFrame original
df_homicidios.loc[Coordenada_X0.index, 'pos x'] = Coordenada_X0['pos x']

C:\Users\davin\AppData\Local\Temp\ipykernel_20000\1853481703.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Coordenada_X0['pos x'] = Coordenada_X0.apply(imputar_pos_x, axis=1)


a continuación se realiza la imputación para el eje y:

In [11]:
# Filtrar las filas donde 'pos y es igual a '.'
Coordenada_Y0 = df_homicidios[df_homicidios['pos y'] == '.']

# Obtener un DataFrame de referencia con valores de 'pos y' que no son '.'
Referencia_Y0 = df_homicidios[df_homicidios['pos y'] != '.']

# Imputar los valores de 'pos y' basándose en la comuna de la Referencia_Y0
def imputar_pos_y(row):
    filtro = (Referencia_Y0['COMUNA'] == row['COMUNA'])
    if filtro.any():
        return Referencia_Y0[filtro]['pos y'].iloc[0]
    else:
        return row['pos y']

Coordenada_Y0['pos y'] = Coordenada_Y0.apply(imputar_pos_y, axis=1)

# Actualizar los valores en el DataFrame original
df_homicidios.loc[Coordenada_Y0.index, 'pos y'] = Coordenada_Y0['pos y']

C:\Users\davin\AppData\Local\Temp\ipykernel_20000\1002031743.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Coordenada_Y0['pos y'] = Coordenada_Y0.apply(imputar_pos_y, axis=1)


Cambiar los registros en coordenadas "." con "SD", son solo dos registros que se manterndrán porque suma en los indicadores, con el campo "SD" puede filtrarse (omitirse) estos datos desde el dashboard:

In [12]:
df_homicidios.loc[df_homicidios['pos y'] == '.', 'pos y'] = 'SD'
df_homicidios.loc[df_homicidios['pos x'] == '.', 'pos x'] = 'SD'

#### 3.1.3. Revisión de la dirección normalizada que presenta datos nulos:

In [13]:
# Filtrar las filas donde 'pos y es igual a '.'
dirección_SD = df_homicidios[df_homicidios['Dirección Normalizada'].isna()]
dirección_SD

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,...,NaN,NaN,NaN,13,Point (. .),-58.46743471,-34.53476874,MOTO-SD,MOTO,SD
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,...,NaN,NaN,NaN,4,Point (. .),-58.36882665,-34.64291993,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,SD,Point (. .),SD,SD,PEATON-SD,PEATON,SD
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,...,NaN,NaN,NaN,9,Point (. .),-58.50877521,-34.66977709,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,...,NaN,NaN,NaN,7,Point (. .),-58.45157703,-34.63063795,AUTO-AUTO,AUTO,AUTO
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,...,NaN,NaN,NaN,14,Point (. .),-58.40021959,-34.57964781,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,...,NaN,NaN,NaN,14,Point (. .),-58.40021959,-34.57964781,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,...,NaN,NaN,NaN,4,Point (. .),-58.36882665,-34.64291993,MOTO-CARGAS,MOTO,CARGAS


In [14]:
df_homicidios['Dirección Normalizada'].nunique()

635

la dirección en su mayoría representa valores unicos y esto no serviría mucho en clasificación por zonas de los accidentes, así que se decide usar coordenadas y comunas como referecias. No se imputarán direcciones con lugar del hecho, así esto sea posible porque dirección será elimnado.

#### 3.1.4. Eliminar columna innecesarias y editar tipos de datos correspondientes:

In [15]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['N_VICTIMAS', 'AAAA', 'MM', 'DD', 'HH', 'LUGAR_DEL_HECHO', 'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'XY (CABA)', 'PARTICIPANTES']

# Eliminar las columnas especificadas
df_homicidios = df_homicidios.drop(columns=columnas_a_eliminar)

# Mostrar el DataFrame actualizado
df_homicidios

,ID,FECHA,HORA,TIPO_DE_CALLE,COMUNA,pos x,pos y,VICTIMA,ACUSADO
0,2016-0001,2016-01-01,04:00:00,AVENIDA,8,-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,2016-01-02,01:15:00,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,2016-01-03,07:00:00,AVENIDA,1,-58.39040293,-34.63189362,MOTO,AUTO
3,2016-0004,2016-01-10,00:00:00,AVENIDA,8,-58.46503904,-34.68092974,MOTO,SD
4,2016-0005,2016-01-21,05:20:00,AVENIDA,1,-58.38718297,-34.62246630,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...
691,2021-0093,2021-12-13,17:10:00,AVENIDA,7,-58.43353773,-34.64561636,MOTO,AUTO
692,2021-0094,2021-12-20,01:10:00,AUTOPISTA,9,-58.46739825,-34.65117757,MOTO,AUTO
693,2021-0095,2021-12-30,00:43:00,AVENIDA,11,-58.47293407,-34.61984745,MOTO,CARGAS
694,2021-0096,2021-12-15,10:30:00,AVENIDA,9,-58.47066794,-34.65021673,AUTO,CARGAS


In [16]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   FECHA          696 non-null    datetime64[ns]
 2   HORA           696 non-null    object        
 3   TIPO_DE_CALLE  696 non-null    object        
 4   COMUNA         696 non-null    object        
 5   pos x          696 non-null    object        
 6   pos y          696 non-null    object        
 7   VICTIMA        696 non-null    object        
 8   ACUSADO        696 non-null    object        
dtypes: datetime64[ns](1), object(8)
memory usage: 49.1+ KB


#### 3.2. Exportar a CSV en la carpeta de Datos_Limpios

In [17]:
df_homicidios.to_csv('Datos_limpios/Hechos_Homicidios_CABA.csv',index=False)

#### 4. Crear Datafrome de la hoja de "Víctimas" del archivo de excel "Homicidios":

In [18]:
# crear dataframe
df_victimas = xl.parse('VICTIMAS',skiprows=0)

# Reiniciar los índices después de la eliminación
df_victimas.reset_index(drop=True, inplace=True)

##### 4.1. Revisamos la calidad de los datos y datos nulos:

In [19]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [20]:
# Contar los valores "SD" en todas las columnas
conteo_SD_por_columna = (df_victimas == "SD").sum()

# Mostrar el resultado
print(conteo_SD_por_columna)

ID_hecho                0
FECHA                   0
AAAA                    0
MM                      0
DD                      0
ROL                    11
VICTIMA                 9
SEXO                    6
EDAD                   53
FECHA_FALLECIMIENTO    68
dtype: int64


#### 4.2. Elimar columnas innecesarias y cambiar tipo correspondiente en fecha:

In [21]:
# Lista de columnas a eliminar
columnas_vic_eliminar = ['AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO']

# Eliminar las columnas especificadas
df_victimas = df_victimas.drop(columns=columnas_vic_eliminar)

# Formato de fecha
df_victimas['FECHA'] = pd.to_datetime(df_victimas['FECHA'], errors='coerce')

# Mostrar el DataFrame actualizado
df_victimas

,ID_hecho,FECHA,ROL,VICTIMA,SEXO,EDAD
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29
...,...,...,...,...,...,...
712,2021-0092,2021-12-12,PEATON,PEATON,FEMENINO,50
713,2021-0093,2021-12-13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18
714,2021-0094,2021-12-20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43
715,2021-0095,2021-12-30,CONDUCTOR,MOTO,MASCULINO,27


In [22]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ID_hecho  717 non-null    object        
 1   FECHA     717 non-null    datetime64[ns]
 2   ROL       717 non-null    object        
 3   VICTIMA   717 non-null    object        
 4   SEXO      717 non-null    object        
 5   EDAD      717 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 33.7+ KB


#### 4.2. Exportar archivo a la carpeta "Datos_Limpios":

In [23]:
df_victimas.to_csv('Datos_limpios/Victimas_Homicidios_CABA.csv',index=False)